# Solição para cotar e comparar preço de produtos na web

- Pode ser agendado como um executável Python e Agendador de tarefas do Windows
- Pode ser executando com um While True

### Importando módulo e lista de produtos

In [108]:
import pandas as pd
from selenium import webdriver
import re
import time
import yagmail


produto = pd.read_excel('Produtos.xlsx').fillna('-')
display(produto)


,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/Iphone-Apple-Preto-1...,https://www.americanas.com.br/produto/16113180...,https://www.magazineluiza.com.br/iphone-11-app...,5297,-
1,"iPhone 12 Apple 64GB Preto 6,1” 12MP iOS",https://www.amazon.com.br/Apple-iPhone-12-128-...,https://www.americanas.com.br/produto/35910240...,https://www.magazineluiza.com.br/iphone-12-min...,2900,-


In [109]:
def tratar_texto(texto):
     return float(re.sub(r'[A-Z$\.]','', texto).replace(',','.'))

In [110]:
#Desloca a tela
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options)

driver = webdriver.Chrome()

enviar_email = False
desconto_min = 0.1

for i, linha in produto.iterrows():
    
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_css_selector('span.apexPriceToPay').text
        preco_amazon = tratar_texto(preco_amazon)
    except:
        print('Produto {} não encontrado'.format(linha['Link Produto']))
        preco_amazon = linha['Preço Original'] * 3   
    time.sleep(1)

    #Entrar no site da L Americanas
    driver.get(linha['Lojas Americanas'])
    try:
        preco_americanas = driver.find_element_by_css_selector('div.priceSales').text
        preco_americanas = tratar_texto(preco_americanas)
    except:
        print('Produto {} não encontrado'.format(linha['Link Produto']))
        preco_americanas = linha['Preço Original'] * 3   
    time.sleep(1)

    #Pegar o preco da MGLU
    driver.get(linha['Magazine Luiza'])
    try:     
        preco_magalu = driver.find_element_by_css_selector('span.price-template__text').text
        preco_magalu = tratar_texto(preco_magalu)
    except:
        print('Produto {} não encontrado'.format(linha['Link Produto']))
        preco_magalu = linha['Preço Original'] * 3   
    time.sleep(1)
    
    preco_original = linha['Preço Original']
    lista_precos = [(preco_amazon, 'Amazon'),(preco_americanas, 'Lojas Americanas'),(preco_magalu, 'Magazine Luiza'),(preco_original, 'Preço Original')]
    lista_precos.sort()
    
    #Preenche linha com preço e nome do local na planilha
    produto.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produto.loc[i, 'Local'] = lista_precos[0][1]
    
    if lista_precos[0][0] <= preco_original * (1-desconto_min):
        enviar_email = True
        
#SALVAR ARQUIVO
produto.to_excel('Produtos.xlsx')

C:\Users\dougl\AppData\Local\Temp/ipykernel_2116/1368604841.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  preco_amazon = driver.find_element_by_css_selector('span.apexPriceToPay').text
C:\Users\dougl\AppData\Local\Temp/ipykernel_2116/1368604841.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  preco_americanas = driver.find_element_by_css_selector('div.priceSales').text
C:\Users\dougl\AppData\Local\Temp/ipykernel_2116/1368604841.py:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  preco_magalu = driver.find_element_by_css_selector('span.price-template__text').text


Produto iPhone 12 Apple 64GB Preto 6,1” 12MP iOS não encontrado


In [111]:
usuario = yagmail.SMTP(user='douglasmarketingbr@gmail.com', password='Bilhao50')
if enviar_email:
    #ENVIAR EMAIL
    destinatario = 'carvalhodouglaspereira@gmail.com'
    assunto = f'Produto(s) com mais de {desconto_min:.0%} de desconto'
    #filtrar tabela
    tabela_filtrada = produto.loc[produto['Preço Atual'] <= produto['Preço Original']*(1-desconto_min), :]
    conteudo = f"""

    <p>Esses são os produtos com mais de {desconto_min:.0%} de desconto</p>
    {tabela_filtrada.to_html()}

    """
    usuario.send(to=destinatario, subject=assunto, contents=conteudo)
    
driver.quit()